In [ ]:
from diffusers import DiffusionPipeline
from diffusers import UNet2DModel 

In [ ]:
repo_id = "google/ddpm-cat-256"

# Load the model
model = UNet2DModel.from_pretrained(repo_id, use_safetensors=True)

In [ ]:
print(model)

In [ ]:
# check all the blocks with 'up' or 'down' in the name
up = []
down = []
for name, blocks in model.named_children():
    if 'up' in name:
        up.append(blocks)
    if 'down' in name:
        down.append(blocks)

In [ ]:
print(up)

In [ ]:
print(model.config)
# those parameters are used to initialize the model, so they are not trainable

In [ ]:
import torch

torch.manual_seed(0)

noisy_sample = torch.randn(1, model.config.in_channels, model.config.sample_size, model.config.sample_size)
noisy_sample.shape

In [ ]:
with torch.no_grad():
    noisy_residual = model(sample=noisy_sample, timestep=2).sample

In [ ]:
from diffusers import DDPMScheduler

scheduler = DDPMScheduler.from_pretrained(repo_id)
print(scheduler)

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
print(pipeline) 

In [ ]:
pipeline.to("cuda")
image = pipeline("Image of a pokemon").image[0]
image

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "shibing624/asian-role"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.safety_checker = lambda images, **kwargs: (images, False)

prompt = "1girl"
negative_prompt = """(((simple background))),monochrome ,lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, ugly, pregnant,vore,duplicate,morbid,mut ilated,tran nsexual, hermaphrodite,long neck,mutated hands,poorly drawn hands,poorly drawn face,mutation, deformed, (((missing arms))),(((missing legs))), (((extra arms))),(((extra legs))),pubic hair, plump,bad legs,error legs, bad feet, loli, little girl"""

image = pipe(prompt, height=512, width=512, num_inference_steps=30, guidance_scale=6, 
    negative_prompt=negative_prompt, num_images_per_prompt=1).images[0]

image.save("./1girl.png")